In [1]:
import random
import time
import json
import re
import collections
import string
from pprint import pprint

from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm

import urllib.request
from urllib.error import URLError, HTTPError

from rupo.api import Engine
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search

Using TensorFlow backend.


In [2]:
import warnings
warnings.filterwarnings('ignore')

**[Python library](https://github.com/IlyaGusev/rupo) for analysis and generation of poems in Russian**

In [3]:
engine = Engine(language='ru')

In [4]:
engine.load('stress_ru_LSTM64_dropout0.2_acc99_wer8.h5', 'dict/zaliznyak.txt')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


**Коллекция [стихотворений](http://www.100bestpoems.ru)**

In [5]:
t = random.randint(0, 3)

In [6]:
main_page = 'http://www.100bestpoems.ru/index.php'

In [7]:
def get_page_html(page_url):
    '''Получение HTML-кода страницы'''
    
    try:
        with urllib.request.urlopen(page_url) as response:
            html = response.read().decode('utf-8')
            soup = BeautifulSoup(html, 'html.parser')
    
    except HTTPError as e:
        print('Error code: ', e.code)
        pass
    except URLError as e:
        print('Reason: ', e.reason)
        pass
    
    return soup

In [8]:
def get_data(soups):
    '''Извлечение метаинформации и текста стихотворений'''
    
    titles, authors, dates, labels, languages, texts = [], [], [], [], [], []
    
    for soup in soups:
    
        title = soup.find('h1', attrs={'itemprop':'name'})
        if title:
            p_title = title.text.strip()
        else:
            p_title = None

        titles.append(p_title)

        author = soup.find('span', attrs={'itemprop':'name'})
        if author:
            p_author = author.text.strip()
        else:
            p_author = None

        authors.append(p_author)

        date = soup.find('td', attrs={'itemprop':'datePublished'})
        if date:
            p_date = date.text.strip().replace(' ', '').replace(',','-')
        else:
            p_date = None

        dates.append(int(p_date[-4:]))
        
        language = re.findall('<a href=".*?" title=".*?">(.*?)</a>', str(soup), re.DOTALL)
        if language:
            p_language = language[4]
        else:
            p_language = None

        languages.append(p_language)

        main_text = soup.find('span', attrs={'itemprop':'citation'})
        if main_text:
            p_text = main_text.text.strip().replace('\r', '\n').replace('\t', '')
        else:
            p_text = None

        texts.append(p_text)
    
    return titles, authors, dates, languages, texts

In [9]:
def get_stats(soups):
    '''Извлечение статистики для стихотворений'''
    
    positions, voters_num, avg_scores, total_scores = [], [], [], []
    
    for soup in soups:
        
        spans = soup.find_all('span', attrs={'class':'t-dgray'})
        if spans:
            position = spans[0].find('b').text.strip()
            total_score = spans[1].find('b').text.strip()
        else:
            position = None
            total_score = None

        positions.append(int(position))
        total_scores.append(int(total_score))
        
        voters = soup.find('span', attrs={'itemprop':'ratingCount'})
        if voters:
            p_voters = voters.text.strip()
        else:
            p_voters = None

        voters_num.append(int(p_voters))
        
        avg_score = soup.find('span', attrs={'itemprop':'ratingValue'})
        if avg_score:
            p_avg_score = avg_score.text.strip()
        else:
            p_avg_score = None
        
        avg_scores.append(float(p_avg_score))
        
    return positions, voters_num, avg_scores, total_scores

In [10]:
def item_info(href):
    '''Извлечение id страницы стихотворения (/item_info.php?id=348)'''
    
    return href and re.compile('item_info').search(href)

In [11]:
main_page_soup = get_page_html(main_page)
ids = main_page_soup.find_all(href=item_info)

In [12]:
def get_ids(ids):
    '''Извлечение id страницы стихотворения (/item_info.php?id=348)'''
    
    endings = []
    
    for element in ids:
        endings.append(element.get('href'))
    
    return endings

In [13]:
def get_poems_pages(endings):
    '''Получение HTML-кода страниц со стихотворениями'''
    poems_htmls = []
    
    urls = ['http://www.100bestpoems.ru/' + end for end in endings]
    time.sleep(t)

    for url in urls:
        poems_htmls.append(get_page_html(url))
        
    return poems_htmls, urls

In [14]:
endings = get_ids(ids[:100])

In [15]:
poems_pages, urls = get_poems_pages(endings)

In [16]:
titles, authors, dates, languages, texts = get_data(poems_pages)
positions, voters_num, avg_scores, total_scores = get_stats(poems_pages)

In [18]:
def write_file(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        for item in data:
            f.write("%s\n" % item)

In [19]:
# meta
write_file('titles.txt', titles)
write_file('authors.txt', authors)
write_file('dates.txt', dates)
write_file('languages.txt', languages)
write_file('texts.txt', texts)

In [20]:
# stats
write_file('positions.txt', positions)
write_file('voters_num.txt', voters_num)
write_file('avg_scores.txt', avg_scores)
write_file('total_scores.txt', total_scores)

**Анализ стихотворений**

In [21]:
def get_poetic_feet(poems):
    '''Определение стихотворного размера'''
    
    poetic_feet = []
    
    for poem in tqdm(poems):
        poetic_feet.append(engine.classify_metre(poem))
    
    return poetic_feet

In [22]:
feet = get_poetic_feet(texts)

In [23]:
for idx, element in enumerate(feet):
    if element == 'iambos':
        feet[idx] = 'ямб'
    if element == 'choreios':
        feet[idx] = 'хорей'
    if element == 'daktylos':
        feet[idx] = 'дактиль'
    if element == 'amphibrachys':
        feet[idx] = 'амфибрахий'
    if element == 'anapaistos':
        feet[idx] = 'анапест'
    if element == 'dolnik3':
        feet[idx] = 'дольник3'
    if element == 'dolnik2':
        feet[idx] = 'дольник2'
    if element == 'taktovik3':
        feet[idx] = 'тактовик3'
    if element == 'taktovik2':
        feet[idx] = 'тактовик2'

In [24]:
counter = collections.Counter(feet)
print(counter)

Counter({'ямб': 37, 'хорей': 28, 'анапест': 19, 'дольник3': 9, 'амфибрахий': 5, 'дактиль': 2})


In [25]:
write_file('feet.txt', feet)

In [26]:
exclude = set(string.punctuation)

In [27]:
def clean_texts(texts):
    cleaned = []
    
    for text in texts:
        cleaned.append(''.join(ch for ch in text if ch not in exclude))
    
    return cleaned

In [28]:
cleaned = clean_texts(texts)

In [29]:
strings = []
for text in cleaned:
    s = [s.lstrip().replace('x', 'х') for s in text.split('\n')]
    strings.append(s)

In [30]:
idxs = []
i = 0
for st in strings:
    lst = [i] * len(st)
    idxs.append(lst)
    i += 1

In [31]:
def get_stress(texts):
    '''Определение ударения'''
    strings = []
    stresses = []
    
    for text in texts:
        s = [s.lstrip().replace('x', 'х') for s in text.split('\n')]
        strings.append(s)

    for s in strings:
        s_result = []
        for i in s:
            result = []
            for word in i.split():
                try:
                    result.append(word[:engine.get_stresses(word)[0]].lower() + \
                                    word[engine.get_stresses(word)[0]:].capitalize())
                except:
                    result.append(word.lower())
            
            s_result.append(result)
        
        stresses.append([' '.join(s) for s in s_result])
        
    return stresses

In [32]:
stresses = get_stress(cleaned)

In [33]:
def get_syllables(texts):
    syllables = []
    
    for text in texts:
        s_result = []
        for t in text:
            result = []
            for word in t.split():
                if engine.get_word_syllables(word) != []:
                    result.append('-'.join(engine.get_word_syllables(word)))
                else:
                    result.append(word.lower())
            
            s_result.append(result)
    
        syllables.append([' '.join(s) for s in s_result])
                
    return syllables

In [34]:
syllables = get_syllables(stresses)

In [35]:
psyllables = []

for s in syllables:
    psyllables.append('\n'.join(s))

In [36]:
write_file('stresses.txt', stresses)
write_file('syllables.txt', syllables)

#### База данных

In [37]:
es = Elasticsearch()
es

<Elasticsearch([{}])>

In [39]:
poems_json = [{"url": url,
               "position" : position,
               "voters" : n_voters,
               "total score" : total_score,
               "average score" : avg_score,
               "title": title,
               "author": author,
               "date": date,
               "language": language,
               "text": text,
               "stresses": stress,
               "syllables": syllable,
               "foot" : foot
              } 
              for url, position, n_voters, total_score, avg_score, title, author, date, language, text, stress, syllable, foot
              in zip(urls, positions, voters_num, total_scores, avg_scores, titles, authors, dates, languages, texts, stresses, syllables, feet)]

In [40]:
with open('poems.json', 'w') as outfile:
    json.dump(poems_json, outfile)

In [41]:
# pprint(poems_json[12])

In [42]:
es.indices.delete("all")
es.indices.delete("poems")
es.indices.delete("stats")
es.indices.delete("analysis")

{'acknowledged': True}

In [43]:
# all info
es.indices.create(index="all")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'all'}

In [44]:
# texts + meta-info + foot
es.indices.create(index="poems")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'poems'}

In [45]:
# stats
es.indices.create(index="stats")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'stats'}

In [46]:
# analysis
es.indices.create(index="analysis")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'analysis'}

In [47]:
mapit_all = {"info" : {"properties" : {"url" : {"type" : "text"},
                                       "title" : {"type" : "text", "analyzer" : "russian"},
                                       "author" : {"type" : "text", "analyzer" : "russian"},
                                       "year" : {"type" : "integer"},
                                       "language" : {"type" : "text", "analyzer" : "russian"},
                                       "text" : {"type" : "text", "analyzer" : "russian"},
                                       "syllables" : {"type" : "text", "analyzer" : "russian"},
                                       "foot" : {"type" : "text", "analyzer" : "russian"},
                                       "position" : {"type" : "integer"},
                                       "voters" : {"type" : "integer"},
                                       "avg_score" : {"type" : "float"},
                                       "total_score" : {"type" : "integer"}}}}

es.indices.put_mapping(index="all", doc_type='info', body=mapit_all)

{'acknowledged': True}

In [107]:
mapit_poems = {"poem" : {"properties" : {"url" : {"type" : "text"},
                                         "title" : {"type" : "text", "analyzer" : "russian"},
                                         "author" : {"type" : "text", "analyzer" : "russian"},
                                         "year" : {"type" : "integer"},
                                         "language" : {"type" : "text", "analyzer" : "russian"},
                                         "text" : {"type" : "text", "analyzer" : "russian"},
                                         "syllables" : {"type" : "text", "analyzer" : "russian"},
                                         "foot" : {"type" : "text", "fielddata": "true", "analyzer" : "russian"}}}}

es.indices.put_mapping(index="poems", doc_type='poem', body=mapit_poems)

{'acknowledged': True}

In [49]:
# mapit_stats = {"statistics" : {"properties" : {"title" : {"type" : "text", "analyzer" : "russian"},
#                                                "author": {"type" : "text", "analyzer" : "russian"},
#                                                "position" : {"type" : "integer"},
#                                                "voters" : {"type" : "integer"},
#                                                "avg_score" : {"type" : "float"},
#                                                "total_score" : {"type" : "integer"}}}}

# es.indices.put_mapping(index="stats", doc_type='statistics', body=mapit_stats)

{'acknowledged': True}

In [50]:
# mapit_analysis = {"ana" : {"properties" : {"text" : {"type" : "text", "analyzer" : "russian"},
#                                            "stresses" : {"type" : "text", "analyzer" : "russian"},
#                                            "syllables" : {"type" : "text", "analyzer" : "russian"}}}}

# es.indices.put_mapping(index="analysis", doc_type='ana', body=mapit_analysis)

{'acknowledged': True}

In [51]:
es.indices.get_alias("*")

{'analysis': {'aliases': {}},
 'poems': {'aliases': {}},
 'all': {'aliases': {}},
 'stats': {'aliases': {}}}

In [52]:
es.indices.get_mapping(index="poems", doc_type='poem')

{'poems': {'mappings': {'poem': {'properties': {'author': {'type': 'text',
      'analyzer': 'russian'},
     'foot': {'type': 'text', 'analyzer': 'russian'},
     'language': {'type': 'text', 'analyzer': 'russian'},
     'syllables': {'type': 'text', 'analyzer': 'russian'},
     'text': {'type': 'text', 'analyzer': 'russian'},
     'title': {'type': 'text', 'analyzer': 'russian'},
     'url': {'type': 'text'},
     'year': {'type': 'integer'}}}}}}

#### Заполнение базы

In [53]:
for i in range(len(poems_json)):
    es.index(index="all", doc_type="info", body={"url": poems_json[i]["url"],
                                                 "title": poems_json[i]["title"],
                                                 "author": poems_json[i]["author"],
                                                 "year": poems_json[i]["date"],
                                                 "language": poems_json[i]["language"],
                                                 "text": poems_json[i]["text"],
                                                 "syllables": poems_json[i]["syllables"],
                                                 "foot": poems_json[i]["foot"],
                                                 "position": poems_json[i]["position"],
                                                 "voters": poems_json[i]["voters"],
                                                 "avg_score": poems_json[i]["average score"],
                                                 "total_score": poems_json[i]["total score"]
                                                  })

In [54]:
for i in range(len(poems_json)):
    es.index(index="poems", doc_type="poem", body={"url": poems_json[i]["url"],
                                                   "title": poems_json[i]["title"],
                                                   "author": poems_json[i]["author"],
                                                   "year": poems_json[i]["date"],
                                                   "language": poems_json[i]["language"],
                                                   "text": poems_json[i]["text"],
                                                   "syllables": poems_json[i]["syllables"],
                                                   "foot": poems_json[i]["foot"]
                                                  })

In [55]:
# for i in range(len(poems_json)):
#     es.index(index="stats", doc_type="statistics", body={"title": poems_json[i]["title"],
#                                                          "author": poems_json[i]["author"],
#                                                          "position": poems_json[i]["position"],
#                                                          "voters": poems_json[i]["voters"],
#                                                          "avg_score": poems_json[i]["average score"],
#                                                          "total_score": poems_json[i]["total score"]})

In [56]:
# for i in range(len(poems_json)):
#     es.index(index="analysis", doc_type="ana", body={"text" : poems_json[i]["text"],
#                                                      "stresses" : poems_json[i]["stresses"],
#                                                      "syllables" : poems_json[i]["syllables"]})

#### Поиск

### 1.

In [57]:
res = es.search(index="poems", body={"query": {"match": {'title': 'любить'}}})
res["hits"]["hits"][1]

{'_index': 'poems',
 '_type': 'poem',
 '_id': 'AfalsmkB3Jb2oNdYXVrZ',
 '_score': 2.8944502,
 '_source': {'url': 'http://www.100bestpoems.ru/item_info.php?id=7718',
  'title': 'Я вас любил: любовь еще, быть может...',
  'author': 'Александр Пушкин',
  'year': 1829,
  'language': 'Русский',
  'text': 'Я вас любил: любовь еще, быть может,\nВ душе моей угасла не совсем;\nНо пусть она вас больше не тревожит;\nЯ не хочу печалить вас ничем.\nЯ вас любил безмолвно, безнадежно,\nТо робостью, то ревностью томим;\nЯ вас любил так искренно, так нежно,\nКак дай вам бог любимой быть другим.',
  'syllables': ['Я вАс лю-бИл лю-бОвь е-ще бЫть мО-жет',
   'в дУ-ше мо-Ей у-гА-сла нЕ сО-всем',
   'нО пУсть о-нА вАс бОль-ше нЕ тре-вО-жит',
   'Я нЕ хо-чУ пе-чА-лить вАс ни-чЕм',
   'Я вАс лю-бИл бе-змОл-вно бе-зна-дЕ-жно',
   'тО рО-бо-стью тО рЕ-вно-стью то-мИм',
   'Я вАс лю-бИл тАк И-скре-нно тАк не-жнО',
   'кАк дАй вАм бОг лю-бИ-мой бЫть дру-гИм'],
  'foot': 'ямб'}}

In [58]:
res = es.search(index="poems", body={"query": {"match": {'text': 'зима'}}})
res["hits"]["hits"][1]

for doc in res['hits']['hits']:
    print("Title: '%s'\nAuthor: %s\n\n%s" % (doc['_source']['title'], doc['_source']['author'], doc['_source']['text']))

Title: 'Мне грустно на тебя смотреть...'
Author: Сергей Есенин

Мне грустно на тебя смотреть,
Какая боль, какая жалость!
Знать, только ивовая медь
Нам в сентябре с тобой осталась.
 Чужие губы разнесли
Твое тепло и трепет тела.
Как будто дождик моросит
С души, немного омертвелой.
 Ну что ж! Я не боюсь его.
Иная радость мне открылась.
Ведь не осталось ничего,
Как только желтый тлен и сырость.
 Ведь и себя я не сберег
Для тихой жизни, для улыбок.
Так мало пройдено дорог,
Так много сделано ошибок.
 Смешная жизнь, смешной разлад.
Так было и так будет после.
Как кладбище, усеян сад
В берез изглоданные кости.
 Вот так же отцветем и мы
И отшумим, как гости сада...
Коль нет цветов среди зимы,
Так и грустить о них не надо.
Title: 'Никого не будет в доме'
Author: Борис Пастернак

Никого не будет в доме, 
Кроме сумерек. Один 
Зимний день в сквозном проёме 
Незадёрнутых гардин. 
 Только белых мокрых комьев 
Быстрый промельк маховой. 
Только крыши, снег и, кроме 
Крыш и снега, — никого. 
 И опять за

In [59]:
res = es.search(index="all", body={"query": {"match": {"text": {"query":"скажи дядя", "operator": "and"}}}})

for doc in res['hits']['hits']:
    print("Title: '%s'\nAuthor: %s" % (doc['_source']['title'], doc['_source']['author']))

Title: 'Бородино'
Author: Михаил Лермонтов


___

### 2.

In [60]:
res = es.search(index="all", body={"query": {"range": {"avg_score": {"gte": "2", "lt": "2.5"}}}})

for doc in res['hits']['hits'][:1]:
    print("Title: '%s'\nAuthor: %s\nAverage score: %s" % (doc['_source']['title'], 
                                              doc['_source']['author'], 
                                              doc['_source']['avg_score']
                                             )
         )

Title: 'Я обманывать себя не стану'
Author: Сергей Есенин
Average score: 2.16


```must, should, must_not```

In [61]:
req = {
  "query": {
    "bool": {
      "must": [
        { "match": { "text": "достаю" } },
        { "match": { "text": "паспорт" } }
      ]
    }
  }
}

In [62]:
res = es.search(index="poems", body=req)

for doc in res['hits']['hits']:
    print("Title: '%s'\nAuthor: %s\n\n%s" % (doc['_source']['title'], doc['_source']['author'], doc['_source']['text']))

Title: 'Стихи о советском паспорте'
Author: Владимир Маяковский

Я волком бы 
  выгрыз 
  бюрократизм. 
К мандатам 
  почтения нету. 
К любым 
  чертям с матерями 
  катись 
любая бумажка. 
   Но эту… 
По длинному фронту 
  купе 
  и кают 
чиновник 
  учтивый 
  движется. 
Сдают паспорта, 
  и я 
  сдаю 
   мою 
  пурпурную книжицу. 
К одним паспортам — 
  улыбка у рта. 
К другим — 
  отношение плевое. 
С почтеньем 
  берут, например, 
  паспорта 
с двухспальным 
   английским левою. 
Глазами 
  доброго дядю выев, 
не переставая 
  кланяться, 
берут, 
  как будто берут чаевые, 
паспорт 
  американца. 
На польский — 
   глядят, 
  как в афишу коза. 
На польский — 
  выпяливают глаза 
в тугой 
  полицейской слоновости — 
откуда, мол, 
  и что это за 
 географические новости?
И не повернув 
   головы качан 
и чувств 
  никаких 
  не изведав, 
берут, 
  не моргнув, 
  паспорта датчан 
и разных 
  прочих 
  шведов, 
   И вдруг, 
  как будто 
  ожогом, 
  рот 
скривило 
  господину. 
Это 
  

#### ```MUST + MUST NOT```

In [63]:
req = {
  "query": {
    "bool": {
      "must": [
        { "match": { "author": "Есенин" } }
      ],
      "must_not": [
        { "match": { "foot": "хорей" } },
        { "match": { "foot": "ямб" } }
      ]
    }
  }
}

In [64]:
res = es.search(index="poems", body=req)

for doc in res['hits']['hits'][:1]:
    print("Title: '%s'\nAuthor: %s\nFoot: %s\n\n%s" % (doc['_source']['title'], 
                                            doc['_source']['author'], 
                                            doc['_source']['foot'],
                                            doc['_source']['text']))

Title: 'Пой же, пой. На проклятой гитаре'
Author: Сергей Есенин
Foot: анапест

Пой же, пой. На проклятой гитаре
Пальцы пляшут твои в полукруг.
Захлебнуться бы в этом угаре,
Мой последний, единственный друг.
 Не гляди на ее запястья
И с плечей ее льющийся шелк.
Я искал в этой женщине счастья,
А нечаянно гибель нашел.
 Я не знал, что любовь — зараза,
Я не знал, что любовь — чума.
Подошла и прищуренным глазом
Хулигана свела с ума.
 Пой, мой друг. Навевай мне снова
Нашу прежнюю буйную рань.
Пусть целует она другова,
Молодая, красивая дрянь.
 Ах, постой. Я ее не ругаю.
Ах, постой. Я ее не кляну.
Дай тебе про себя я сыграю
Под басовую эту струну.
 Льется дней моих розовый купол.
В сердце снов золотых сума.
Много девушек я перещупал,
Много женщин в углах прижимал.
 Да! есть горькая правда земли,
Подсмотрел я ребяческим оком:
Лижут в очередь кобели
Истекающую суку соком.
 Так чего ж мне ее ревновать.
Так чего ж мне болеть такому.
Наша жизнь — простыня да кровать.
Наша жизнь — поцелуй да в омут

In [65]:
req = {
  "query": {
    "bool": {
      "must": [
        { "match": { "text": "солнце" } },
        { "range": { "avg_score": { "gte": "2", "lt": "2.5" } } }
      ]
    }
  }
}

In [66]:
res = es.search(index = "all", body = req)
for doc in res['hits']['hits'][:2]:
    print("Title: '%s'\nAuthor: %s\nAverage score: %s\n\n%s" % (doc['_source']['title'], 
                                                    doc['_source']['author'], 
                                                    doc['_source']['avg_score'],
                                                    doc['_source']['text']
                                             )
         )

Title: 'Зимнее утро'
Author: Александр Пушкин
Average score: 2.06

Мороз и солнце; день чудесный!
Еще ты дремлешь, друг прелестный —
Пора, красавица, проснись:
Открой сомкнуты негой взоры
Навстречу северной Авроры,
Звездою севера явись!
 Вечор, ты помнишь, вьюга злилась,
На мутном небе мгла носилась;
Луна, как бледное пятно,
Сквозь тучи мрачные желтела,
И ты печальная сидела —
А нынче... погляди в окно:
 Под голубыми небесами
Великолепными коврами,
Блестя на солнце, снег лежит;
Прозрачный лес один чернеет,
И ель сквозь иней зеленеет,
И речка подо льдом блестит.
 Вся комната янтарным блеском
Озарена. Веселым треском
Трещит затопленная печь.
Приятно думать у лежанки.
Но знаешь: не велеть ли в санки
Кобылку бурую запречь?
 Скользя по утреннему снегу,
Друг милый, предадимся бегу
Нетерпеливого коня
И навестим поля пустые,
Леса, недавно столь густые,
И берег, милый для меня.
Title: 'Парус'
Author: Михаил Лермонтов
Average score: 2.06

Белеет парус одинокой
В тумане моря голубом!..
Что ищет о

In [67]:
req = {
  "query": {
    "bool": {
      "should": [
        { "match": { "text": "мир" } },
        { "range": { "year": { "gte": "1995", "lt": "2000" } } }
      ]
    }
  }
}

In [68]:
res = es.search(index = "all", body = req)

for doc in res['hits']['hits'][:2]:
    print("Title: '%s'\nAuthor: %s\nYear: %s\n\n%s" % (doc['_source']['title'], 
                                                    doc['_source']['author'], 
                                                    doc['_source']['year'],
                                                    doc['_source']['text']
                                             )
         )

Title: 'Лебединая верность'
Author: Андрей Дементьев
Year: 1975

Над землей летели лебеди
Солнечным днем.
Было им светло и радостно
В небе вдвоем,
И земля казалась ласковой
Им в этот миг..
Вдруг по птицам кто-то выстрелил,
И вырвался крик:
 Что с тобой, моя любимая?
Отзовись скорей.
Без любви твоей
Небо все грустней.
Где же ты, моя любимая?
Возвратись скорей,
Красотой своею нежной
Сердце мне согрей.
 В небесах искал подругу он,
Звал из гнезда,
Но молчанием ответила
Птице беда.
Улететь в края далекие 
Лебедь не смог,
Потеряв подругу верную,
Он стал одинок.
 Ты прости меня, любимая,
За чужое зло,
Что мое крыло
Счастье не спасло.
Ты прости меня, любимая,
Что весенним днем
В небе голубом, как прежде,
Нам не быть вдвоем.
 И была непоправимою
Эта беда,
Что с любимою не встретится
Он никогда.
Лебедь вновь поднялся к облаку,
Песню прервал. 
И, сложив бесстрашно крылья, 
На землю упал.
 Я хочу, чтоб жили лебеди,
И от белых стай,
И от белых стай
Мир добрее стал.
 Пусть летят по небу лебеди
Над з

In [69]:
req = {
  "query": {
    "bool": {
      "must": [
        { "match": { "text": "любовь" } },
        { "range": { "year": { "gte": "1970", "lt": "2000" } } }
      ]
    }
  }
}

In [70]:
res = es.search(index = "all", body = req)
for doc in res['hits']['hits']:
    print("Title: '%s'\nAuthor: %s\nYear: %s\n\n%s" % (doc['_source']['title'],
                                                       doc['_source']['author'],
                                                       doc['_source']['year'],
                                                       doc['_source']['text']
                                                      )
         )

Title: 'Любовь и разлука'
Author: Булат Окуджава
Year: 1982

Ещё он не сшит, твой наряд подвенечный,
и хор в нашу честь не споёт...
А время торопит - возница беспечный, -
и просятся кони в полёт.
 Ах, только бы тройка не сбилась бы с круга,
не смолк бубенец под дугой...
Две вечных подруги - любовь и разлука -
не ходят одна без другой.
 Мы сами раскрыли ворота, мы сами
счастливую тройку впрягли,
и вот уже что-то сияет пред нами,
но что-то погасло вдали.
 Святая наука - расслышать друг друга
сквозь ветер, на все времена...
Две странницы вечных - любовь и разлука -
поделятся с нами сполна.
 Чем дольше живём мы, тем годы короче,
тем слаще друзей голоса.
Ах, только б не смолк под дугой колокольчик,
глаза бы глядели в глаза.
 То берег - то море, то солнце - то вьюга,
то ангелы - то воронье...
Две верных дороги - любовь и разлука -
проходят сквозь сердце моё.


In [71]:
req = {
  "query": { 
    "bool": { 
      "must": [
        { "match": { "author": "Высоцкий" }}, 
        { "match": { "text": "яблоко" }}  
      ],
      "filter": [ 
        { "term":  { "foot": "анапест" }}, 
        { "range": { "year": { "gte": "1970" }}} 
      ]
    }
  }
}

In [72]:
res = es.search(index = "all", body = req)

for doc in res['hits']['hits']:
    print("Title: '%s'\nAuthor: %s\nYear: %s\nFoot: %s\n\n%s" % (doc['_source']['title'],
                                                                 doc['_source']['author'],
                                                                 doc['_source']['year'],
                                                                 doc['_source']['foot'],
                                                                 doc['_source']['text']
                                                                  )
         )

Title: 'Райские яблоки'
Author: Владимир Высоцкий
Year: 1977
Foot: анапест

Я когда-то умру - мы когда-то всегда умираем,-
Как бы так угадать, чтоб не сам - чтобы в спину ножом:
Убиенных щадят, отпевают и балуют раем,-
Не скажу про живых, а покойников мы бережем.
 В грязь ударю лицом, завалюсь покрасивее набок,
И ударит душа на ворованных клячах в галоп.
В дивных райских садах наберу бледно-розовых яблок.
Жаль, сады сторожат и стреляют без промаха в лоб.
 Прискакали - гляжу - пред очами не райское что-то:
Неродящий пустырь и сплошное ничто - беспредел.
И среди ничего возвышались литые ворота,
И огромный этап - тысяч пять - на коленях сидел.
 Как ржанет коренной! Я смирил его ласковым словом,
Да репьи из мочал еле выдрал и гриву заплел.
Седовласый старик слишком долго возился с засовом -
И кряхтел и ворчал, и не смог отворить - и ушел.
 И измученный люд не издал ни единого стона,
Лишь на корточки вдруг с онемевших колен пересел.
Здесь малина, братва,- нас встречают малиновым звоном!
Все

In [73]:
req = {
    "aggs" : {
        "max_year" : { "max" : { "field" : "year" } }
    }
}

In [74]:
res = es.search(index="all", body=req)

print(res['aggregations'])

{'max_year': {'value': 2007.0}}


In [97]:
req = {
    "aggs" : {
        "min_year" : { "min" : { "field" : "year" } }
    }
}

In [98]:
res = es.search(index="all", body=req)

print(res['aggregations'])

{'min_year': {'value': 1818.0}}


In [99]:
req = {
    "aggs" : {
        "sum_voters" : { "sum" : { "field" : "voters" } }
    }
}

In [100]:
res = es.search(index="all", body=req)

print(res['aggregations'])

{'sum_voters': {'value': 115577.0}}


In [94]:
# req = {
#     "aggs" : {
#         "all" : {
#             "filter" : { "term": { "author": "Сергей Есенин" } },
#             "aggs" : {
#                 "max_year" : { "max" : { "field" : "year" } }
#             }
#         }
#     }
# }

In [96]:
# res = es.search(index="all", body=req)

# print(res['aggregations'])

{'all': {'doc_count': 0, 'max_year': {'value': None}}}


In [90]:
res = es.search(index='all', body={
        "aggs": {
            "all_years": {
            "terms": { "field": "year" }
            }
        }
    })

In [93]:
print(res['aggregations'])

{'all_years': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 55, 'buckets': [{'key': 1925, 'doc_count': 11}, {'key': 1923, 'doc_count': 7}, {'key': 1924, 'doc_count': 6}, {'key': 1977, 'doc_count': 5}, {'key': 1975, 'doc_count': 4}, {'key': 1837, 'doc_count': 3}, {'key': 1922, 'doc_count': 3}, {'key': 1822, 'doc_count': 2}, {'key': 1825, 'doc_count': 2}, {'key': 1826, 'doc_count': 2}]}}


In [109]:
req = {
    "aggs" : {
        "feet" : {
            "terms" : { "field" : "foot"}
        }
    }
}

In [111]:
res = es.search(index='poems', body=req)
print(res['aggregations'])

{'feet': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': [{'key': 'ямб', 'doc_count': 37}, {'key': 'хор', 'doc_count': 28}, {'key': 'анапест', 'doc_count': 19}, {'key': 'дольник3', 'doc_count': 9}, {'key': 'амфибрах', 'doc_count': 5}, {'key': 'дактил', 'doc_count': 2}]}}
